In [11]:
!sudo apt-get install check
!pip install lyon

import wave
import numpy as np
import os

from lyon.calc import LyonCalc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
check is already the newest version (0.10.0-3build2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


## Single file

In [ ]:
# Read file to get buffer
ifile = wave.open("ti46_0am1s1t0.wav")

sample_rate = ifile.getframerate()

samples = ifile.getnframes()
audio = ifile.readframes(samples)

# Convert buffer to float32 using NumPy                                                                                 
audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
audio_as_np_float64 = audio_as_np_int16.astype(np.float64)

# Normalise float64 array so that values are between -1.0 and +1.0                                                      
max_int16 = 2**15
audio_normalised = audio_as_np_float64 / max_int16
ifile.close()

In [ ]:
audio_normalised.shape

(11264,)

In [ ]:
calc = LyonCalc()
decimation_factor = 64
coch = calc.lyon_passive_ear(audio_normalised, sample_rate, decimation_factor)
print(coch.shape)

(176, 78)


In [ ]:
np.savetxt('array.csv', coch, delimiter=',')

## Multiple Files

In [ ]:
!unzip train.zip

In [8]:
def read_file(name):
  # Read file to get buffer
  ifile = wave.open(name)

  sample_rate = ifile.getframerate()

  samples = ifile.getnframes()
  audio = ifile.readframes(samples)

  # Convert buffer to float32 using NumPy                                                                                 
  audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
  audio_as_np_float64 = audio_as_np_int16.astype(np.float64)

  # Normalise float64 array so that values are between -1.0 and +1.0                                                      
  max_int16 = 2**15
  audio_normalised = audio_as_np_float64 / max_int16
  ifile.close()
  return (audio_normalised, sample_rate)

In [ ]:
for i in ["f", "m"]:
  for j in [1, 2, 3]:
    foldername = "traincsv/" + i + str(j) + "/"
    os.makedirs(os.path.dirname(foldername), exist_ok=True)
    for k in range(10):
      for l in range(10):
        filepath = "train/" + i + str(j) + "/0" + str(k) + i + str(j) + "set" + str(l) + ".wav"
        csvpath = "traincsv/" + i + str(j) + "/0" + str(k) + i + str(j) + "set" + str(l) + ".csv"
        audio_normalised, sample_rate = read_file(filepath)
        calc = LyonCalc()
        decimation_factor = 64
        coch = calc.lyon_passive_ear(audio_normalised, sample_rate, decimation_factor)
        np.savetxt(csvpath, coch, delimiter=',')
        print(csvpath)


In [ ]:
!zip -r traincsv.zip traincsv